# TASK - 2 CREDIT CARD FRAUD DETECTION

# Importing packages

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


# Importing dataset

In [19]:
df = pd.read_csv("fraudTrain.csv")

In [20]:
df.head(2)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0


In [21]:
from copy import deepcopy
actual = deepcopy(df)

# Preprocessing

# label encoding

In [22]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows  
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['merchant']= label_encoder.fit_transform(df['merchant']) 
df['category']=label_encoder.fit_transform(df['category'])
df['first']=label_encoder.fit_transform(df['first'])
df['last']=label_encoder.fit_transform(df['last'])
df['gender']=label_encoder.fit_transform(df['gender'])
df['street']=label_encoder.fit_transform(df['street'])
df['city']=label_encoder.fit_transform(df['city'])
df['state']=label_encoder.fit_transform(df['state'])
df['job']=label_encoder.fit_transform(df['job'])

In [23]:
df.head(2)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,514,8,4.97,162,18,0,568,...,36.0788,-81.1781,3495,370,1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,241,4,107.23,309,157,0,435,...,48.8878,-118.2105,149,428,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0


# finding the correlation among the data

In [24]:
corr = df.corr(method = 'pearson')
corr

,Unnamed: 0,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud
Unnamed: 0,1.000000e+00,0.000386,-0.000972,0.000250,-0.000251,0.000509,0.000121,-0.000945,-0.001067,-0.000423,0.001048,0.000709,0.000602,-0.000676,-0.001678,-7.800295e-07,0.998971,0.000541,-0.000671,-0.004767
cc_num,3.857663e-04,1.000000,0.000055,0.001230,0.001769,-0.027189,0.006908,0.001112,0.046509,0.049188,-0.036373,0.041459,-0.059271,-0.048278,-0.008991,2.916472e-02,0.000354,-0.058942,-0.048252,-0.000981
merchant,-9.718206e-04,0.000055,1.000000,0.032302,-0.002633,-0.001576,-0.000439,-0.000915,0.000718,-0.000294,-0.000424,0.001113,-0.002266,-0.000697,0.001911,1.360387e-03,-0.000999,-0.002263,-0.000673,-0.000536
category,2.495622e-04,0.001230,0.032302,1.000000,0.030867,-0.001931,-0.004319,-0.028259,0.002258,0.000784,0.000211,0.002371,-0.008660,-0.000767,0.009386,3.490427e-03,0.000182,-0.008519,-0.000736,0.020205
amt,-2.511725e-04,0.001769,-0.002633,0.030867,1.000000,-0.003575,-0.004708,0.001034,0.001346,-0.000565,0.001718,0.001843,-0.001926,-0.000187,0.005818,-6.277082e-04,-0.000293,-0.001873,-0.000151,0.219404
first,5.093514e-04,-0.027189,-0.001576,-0.001931,-0.003575,1.000000,-0.056983,0.015197,-0.045369,-0.009211,0.043145,0.004766,0.131869,-0.016475,-0.066607,-3.968275e-02,0.000483,0.130964,-0.016489,-0.003219
last,1.213636e-04,0.006908,-0.000439,-0.004319,-0.004708,-0.056983,1.000000,0.043740,0.005634,0.017553,-0.048161,0.028295,-0.034915,-0.025557,-0.012039,-2.459638e-02,0.000094,-0.034548,-0.025557,-0.000096
gender,-9.450357e-04,0.001112,-0.000915,-0.028259,0.001034,0.015197,0.043740,1.000000,0.042679,0.016039,-0.031794,-0.065951,0.042935,0.050404,-0.028649,-7.990836e-02,-0.000960,0.042645,0.050350,0.007642
street,-1.066586e-03,0.046509,0.000718,0.002258,0.001346,-0.045369,0.005634,0.042679,1.000000,-0.024811,0.014045,-0.053860,-0.012651,0.071328,-0.012530,-2.022876e-02,-0.001089,-0.012517,0.071286,0.001448
city,-4.230686e-04,0.049188,-0.000294,0.000784,-0.000565,-0.009211,0.017553,0.016039,-0.024811,1.000000,-0.043064,0.074757,-0.032044,-0.066703,0.034923,2.044567e-02,-0.000466,-0.031828,-0.066718,-0.002092


In [25]:
corr.iloc[-1,:]

Unnamed: 0   -0.004767
cc_num       -0.000981
merchant     -0.000536
category      0.020205
amt           0.219404
first        -0.003219
last         -0.000096
gender        0.007642
street        0.001448
city         -0.002092
state         0.001730
zip          -0.002162
lat           0.001894
long          0.001721
city_pop      0.002136
job          -0.000093
unix_time    -0.005078
merch_lat     0.001741
merch_long    0.001721
is_fraud      1.000000
Name: is_fraud, dtype: float64

In [26]:
sorted_last_row = corr.iloc[-1, :].sort_values(ascending=True)

# Print the sorted last row
print(sorted_last_row)

unix_time    -0.005078
Unnamed: 0   -0.004767
first        -0.003219
zip          -0.002162
city         -0.002092
cc_num       -0.000981
merchant     -0.000536
last         -0.000096
job          -0.000093
street        0.001448
long          0.001721
merch_long    0.001721
state         0.001730
merch_lat     0.001741
lat           0.001894
city_pop      0.002136
gender        0.007642
category      0.020205
amt           0.219404
is_fraud      1.000000
Name: is_fraud, dtype: float64


In [27]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

# Droping the unwanted datas

In [28]:
columns_to_drop = ['Unnamed: 0','trans_date_trans_time','first','last','zip','lat','long','city_pop','dob','trans_num','unix_time','merch_lat','merch_long']
df1 = df.drop(columns=columns_to_drop)

In [29]:
df1

,cc_num,merchant,category,amt,gender,street,city,state,job,is_fraud
0,2703186189652095,514,8,4.97,0,568,526,27,370,0
1,630423337322,241,4,107.23,0,435,612,47,428,0
2,38859492057661,390,0,220.11,1,602,468,13,307,0
3,3534093764340240,360,2,45.00,1,930,84,26,328,0
4,375534208663984,297,9,41.96,1,418,216,45,116,0
...,...,...,...,...,...,...,...,...,...,...
1296670,30263540414123,499,0,15.56,1,154,330,44,215,0
1296671,6011149206456997,2,1,51.70,1,856,813,20,360,0
1296672,3514865930894695,599,1,105.93,1,158,346,32,308,0
1296673,2720012583106919,509,1,74.90,1,433,471,41,485,0


# spliting dependent and independent data

In [30]:
x = df1.iloc[:10000, :-1]
y = df1.iloc[:10000,-1]

In [31]:
x

,cc_num,merchant,category,amt,gender,street,city,state,job
0,2703186189652095,514,8,4.97,0,568,526,27,370
1,630423337322,241,4,107.23,0,435,612,47,428
2,38859492057661,390,0,220.11,1,602,468,13,307
3,3534093764340240,360,2,45.00,1,930,84,26,328
4,375534208663984,297,9,41.96,1,418,216,45,116
...,...,...,...,...,...,...,...,...,...
9995,6011399591920186,600,3,93.06,0,591,743,19,385
9996,4467456915153312231,241,4,63.62,0,764,792,9,488
9997,4689147265057948423,141,12,5.71,1,257,473,29,234
9998,373905417449658,550,4,231.45,0,557,501,43,340


In [32]:
y

0       0
1       0
2       0
3       0
4       0
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: is_fraud, Length: 10000, dtype: int64

# fitting the model

In [33]:
model = RandomForestClassifier(n_estimators=100,max_depth=15,random_state=0)

In [34]:
pred=model.fit(x,y)

In [35]:

input_data = [[30263540414123, 499, 0, 15.56, 1, 154, 330, 44,215]]

predictions = pred.predict(input_data)


d:\Anaconda\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [36]:
predictions

array([0], dtype=int64)

# Importing test data

In [46]:
test = pd.read_csv("fraudTest.csv")

In [47]:
test.head(2)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0


In [48]:
columns_to_drop = ['Unnamed: 0','trans_date_trans_time','first','last','zip','lat','long','city_pop','dob','trans_num','unix_time','merch_lat','merch_long']
test1 = test.drop(columns=columns_to_drop)

In [49]:
test1

,cc_num,merchant,category,amt,gender,street,city,state,job,is_fraud
0,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,M,351 Darlene Green,Columbia,SC,Mechanical engineer,0
1,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,F,3638 Marsh Union,Altonah,UT,"Sales professional, IT",0
2,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,F,9333 Valentine Point,Bellmore,NY,"Librarian, public",0
3,3591919803438423,fraud_Haley Group,misc_pos,60.05,M,32941 Krystal Mill Apt. 552,Titusville,FL,Set designer,0
4,3526826139003047,fraud_Johnston-Casper,travel,3.19,M,5783 Evan Roads Apt. 465,Falmouth,MI,Furniture designer,0
...,...,...,...,...,...,...,...,...,...,...
555714,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,M,558 Michael Estates,Luray,MO,Town planner,0
555715,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,M,572 Davis Mountains,Lake Jackson,TX,Futures trader,0
555716,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,F,144 Evans Islands Apt. 683,Burbank,WA,Musician,0
555717,4079773899158,fraud_Breitenberg LLC,travel,7.99,M,7020 Doyle Stream Apt. 951,Mesa,ID,Cartographer,0


# Spliting test data

In [50]:
x_test = df1.iloc[:10000, :-1]
y_test = df1.iloc[:10000,-1]

In [51]:
x_test

,cc_num,merchant,category,amt,gender,street,city,state,job
0,2703186189652095,514,8,4.97,0,568,526,27,370
1,630423337322,241,4,107.23,0,435,612,47,428
2,38859492057661,390,0,220.11,1,602,468,13,307
3,3534093764340240,360,2,45.00,1,930,84,26,328
4,375534208663984,297,9,41.96,1,418,216,45,116
...,...,...,...,...,...,...,...,...,...
9995,6011399591920186,600,3,93.06,0,591,743,19,385
9996,4467456915153312231,241,4,63.62,0,764,792,9,488
9997,4689147265057948423,141,12,5.71,1,257,473,29,234
9998,373905417449658,550,4,231.45,0,557,501,43,340


In [52]:
y_test

0       0
1       0
2       0
3       0
4       0
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: is_fraud, Length: 10000, dtype: int64

In [53]:
y_pred = model.predict(x_test)

In [54]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

# Accuracy

In [55]:
from sklearn.metrics import accuracy_score


accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:}")


Accuracy: 1.0
